In [1]:
# import
import torch
from tqdm import tqdm
from transformers import BertConfig, BertForPreTraining
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil
from icecream import ic
from tqdm import tqdm_notebook as tqdm 

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

from torch import nn

from transformers import BertModel

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
dic = {}

with open("../BERT-base_mecab-ipadic-bpe-32k/vocab.txt",  "r", encoding="utf-8" ) as f:
    vocab = f.read()
    for id, word in enumerate(vocab.split('\n')):
        dic[word] = id
        
text = "[CLS] 私 は 犬 が 好き 。 [SEP]"

x = [dic[w] for w in text.split()]

In [3]:
x = torch.LongTensor(x).unsqueeze(0)



In [4]:
bert = BertModel.from_pretrained("cl-tohoku/bert-base-japanese")

In [5]:
a = bert(x)

In [6]:
a[0].shape

torch.Size([1, 8, 768])

In [7]:
a[0][0][3]

tensor([-1.2636e-01, -1.2734e-01,  4.1125e-01, -3.7946e-02,  3.5671e-01,
         5.9472e-01, -1.2080e-02, -2.0193e-01,  6.1377e-01,  2.1312e-01,
        -7.4529e-02,  2.0608e-02, -4.5949e-01,  2.5509e-01, -2.4717e-01,
        -9.4191e-02,  1.1410e-01, -2.2248e-01,  3.5165e-01,  4.6572e-01,
        -3.8768e-01, -8.5532e-02,  2.5526e-01,  2.6166e-01, -3.1824e-01,
         8.1794e-02, -2.4993e-01,  2.9269e-01,  7.9955e-01,  3.5923e-01,
        -1.2338e-01,  6.5812e-03,  3.8730e-01,  2.3091e-01,  1.4636e-01,
         4.6668e-01,  2.6290e-01,  1.6372e-01,  6.9316e-02,  1.4958e-01,
        -5.9775e-01,  2.1966e-01, -1.5152e-01, -7.7528e-03,  2.8760e-02,
         8.2232e-01, -2.1874e-01,  4.2410e-02,  8.8731e-01,  9.9789e-02,
        -6.2738e-01, -4.2279e-01,  9.3123e-03, -3.0431e-01, -2.4775e-01,
        -9.0133e-02,  1.4442e+00,  2.2802e-01,  4.4458e-01, -3.1874e-01,
        -6.9717e-01,  6.2764e-01, -3.7871e-02, -6.9396e-02,  1.3187e-01,
        -1.4630e-01, -9.5384e-02, -4.0134e-02, -1.1

In [8]:
from transformers import BertJapaneseTokenizer

tknz = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')


In [9]:
tknz.tokenize("私は犬が好き")

['私', 'は', '犬', 'が', '好き']

# Masked Language Model

In [10]:
# ids = tknz.encode("私は[MASK]が好き")

In [11]:
# ids

In [12]:
# from transformers import BertForMaskedLM

# model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese')

In [13]:
# a = model(torch.LongTensor(ids).unsqueeze(0))

In [14]:
# mskpos = 3
# b=torch.topk(a[0][0][mskpos], k=5)
# tknz.convert_ids_to_tokens(b[1])

In [15]:
x_train, x_test = [],[]
y_train, y_test = [],[]


In [16]:
train_df = pd.read_csv("../data/train_twitterJSA_data.csv")

test_df = pd.read_csv("../data/test_twitterJSA_data.csv")

In [17]:
# y_train = train_df["label"].tolist()
# y_test = test_df["label"].tolist()

In [18]:
for ind in tqdm(range(len(train_df))):
#     print(ind)
    
    if type(train_df["text"].iloc[ind]  ) != str:
        continue
    tid = tknz.encode(train_df["text"].iloc[ind]) 
    
    if (len(tid)>512):
        tid = tid[:512]
    x_train.append(tid)
    y_train.append(train_df["label"].iloc[ind])
#     break

In [19]:
# net(x_train[1])

# net(torch.LongTensor(x_train[1]).unsqueeze(0).to(device))

In [20]:
for i in range(len(x_train)):
    if (len(x_train[i])) < 2:
        print(x_train[i])


In [21]:
for ind in tqdm(range(len(test_df))):
    
    if type(test_df["text"].iloc[ind]  ) != str:
        continue
    tid = tknz.encode(test_df["text"].iloc[ind]) 
    if (len(tid)>512):
        tid = tid[:512]
    x_test.append(tid)

In [22]:
for ind in tqdm(range(len(test_df))):
    
    if type(test_df["text"].iloc[ind] ) != str:
        continue
    
    tid = tknz.encode(test_df["text"].iloc[ind]) 
    if (len(tid)>512):
        tid = tid[:512]
    x_test.append(tid)
    y_test.append(test_df["label"].iloc[ind])
#     break

In [35]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
class MyDataset(Dataset):
    def __init__(self, xdata, ydata):
        self.data = xdata
        self.label = ydata
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.label[idx]
        return (x,y)

def my_collate_fn(batch):
    images, targets= list(zip(*batch))
    xs = list(images)
    ys = list(targets)
    return xs, ys

In [26]:
batch_size = 2
dataset = MyDataset(x_train,y_train)
dataloader = DataLoader(dataset, batch_size=batch_size,
                        shuffle=True, collate_fn=my_collate_fn)

In [27]:
class DocCls(nn.Module):
    def __init__(self, bert):
        super(DocCls, self).__init__()
        self.bert = bert
        self.cls = nn.Linear(768, 5)
    
    def forward(self, x1, x2):
        bout = self.bert(input_ids=x1, attention_mask=x2)
        bs = len(bout[0])
        h0 = [bout[0][i][0] for i in range(bs)]
        h0 = torch.stack(h0, dim=0)
        return self.cls(h0)
        
    

In [30]:
from torch import optim
net = DocCls(bert)
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)

## 学習

In [36]:
net.train()

for ep in tqdm(range(10)):
    i, lossK=0 ,0.0
    for xs, ys in dataloader:
        xsl, xmsk = [], []
        for k in range(len(xs)):
            tid = xs[k]
            xsl.append(torch.LongTensor(tid))
            xmsk.append(torch.LongTensor([1]*len(tid)))
        
        xsl = pad_sequence(xsl, batch_first = True).to(device)
        xmsk = pad_sequence(xmsk, batch_first = True).to(device)
        outputs = net(xsl, xmsk) 
                        
        dummy_y  =[]

        # 同率１位は、先のほうにする。
                        
        for tmp_y in ys:
            added_flg = 0
            for index, yy in enumerate(tmp_y[1:-1].split(",")):
                if "1" in yy and added_flg == 0:
                    dummy_y.append(index)
                    added_flg = 1

        ys = torch.LongTensor(dummy_y).to(device)
    #         print("out:", out.size(), "y:", y.size())

        loss = criterion(outputs, ys)
        lossK += loss.item()

        if (i % 500 == 0 ):
            print(ep, i, lossK)
            lossK = 0.0

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i+=1
    outfile = "doccls - "+ str(ep) + ".model"
    torch.save(net.state_dict().outfile)
            

0 0 1.4157302379608154
0 0 1.3457483053207397
0 0 1.1765484809875488
0 0 1.221365213394165
0 0 0.8122214674949646
0 0 0.7117689847946167
0 0 2.1324033737182617
0 0 1.3486196994781494
0 0 0.6894455552101135
0 0 0.6975724697113037
0 0 0.42121630907058716
0 0 1.0165293216705322
0 0 1.485957145690918
0 0 0.4497588276863098
0 0 0.25868892669677734
0 0 2.236933708190918
0 0 1.1434364318847656
0 0 0.44342780113220215
0 0 1.9925397634506226
0 0 1.0033652782440186
0 0 0.42250245809555054
0 0 1.0045487880706787
0 0 0.785220205783844
0 0 0.9481338858604431
0 0 0.5350627303123474
0 0 1.2715272903442383
0 0 0.33843499422073364
0 0 1.9862072467803955
0 0 0.35641223192214966
0 0 0.23805725574493408
0 0 0.5009191632270813
0 0 0.9665640592575073
0 0 0.8225530982017517
0 0 0.33830833435058594
0 0 0.454412043094635
0 0 0.48086017370224
0 0 0.14990293979644775
0 0 0.9149404764175415
0 0 0.5012004971504211
0 0 0.7773641347885132
0 0 0.15964579582214355
0 0 0.7739200592041016
0 0 1.007683515548706
0 0 0.733

0 0 0.09440004825592041
0 0 0.24476683139801025
0 0 0.025065898895263672
0 0 0.5226389765739441
0 0 0.19031810760498047
0 0 0.3950803279876709
0 0 0.00995326042175293
0 0 0.23793387413024902
0 0 1.0508171319961548
0 0 0.05453801155090332
0 0 0.020315885543823242
0 0 0.020418643951416016
0 0 0.011231184005737305
0 0 0.019368886947631836
0 0 0.25904715061187744
0 0 0.7503268718719482
0 0 0.9671265482902527
0 0 0.008375406265258789
0 0 1.894463300704956
0 0 0.4731435179710388
0 0 0.4678853750228882
0 0 0.06123852729797363
0 0 0.011127948760986328
0 0 0.41811203956604004
0 0 0.20858681201934814
0 0 0.012208938598632812
0 0 0.018929243087768555
0 0 0.014198064804077148
0 0 0.7542072534561157
0 0 0.4361972212791443
0 0 0.014107942581176758
0 0 0.29563242197036743
0 0 0.5847484469413757
0 0 0.010448932647705078
0 0 0.3058267831802368
0 0 0.5681812763214111
0 0 0.16904962062835693
0 0 0.6188626289367676
0 0 0.00756072998046875
0 0 0.7103608250617981
0 0 0.009717702865600586
0 0 0.0708826780319

0 0 0.4773973822593689
0 0 0.00704193115234375
0 0 0.021405935287475586
0 0 0.007622241973876953
0 0 0.0054340362548828125
0 0 0.7777367830276489
0 0 0.8090416193008423
0 0 0.05835103988647461
0 0 0.13879358768463135
0 0 0.05338311195373535
0 0 0.06200361251831055
0 0 0.45213764905929565
0 0 1.4378535747528076
0 0 0.340485155582428
0 0 0.36495232582092285
0 0 0.5210278630256653
0 0 0.006151914596557617
0 0 1.2819099426269531
0 0 0.9079214930534363
0 0 0.15665102005004883
0 0 0.011182308197021484
0 0 0.22775280475616455
0 0 0.41439223289489746
0 0 0.011170625686645508
0 0 0.006324052810668945
0 0 0.5915436744689941
0 0 0.20690453052520752
0 0 1.2669037580490112
0 0 0.0769416093826294
0 0 0.8772264719009399
0 0 0.8539953231811523
0 0 0.18018269538879395
0 0 0.008525609970092773
0 0 0.16290438175201416
0 0 0.8298788666725159
0 0 0.3432687520980835
0 0 0.005679130554199219
0 0 0.16649866104125977
0 0 0.005570173263549805
0 0 0.4025602340698242
0 0 0.024393796920776367
0 0 0.105646491050720

0 0 0.7277268767356873
0 0 0.05974435806274414
0 0 0.21814346313476562
0 0 0.002308368682861328
0 0 0.002095937728881836
0 0 0.0015473365783691406
0 0 0.612154483795166
0 0 0.002704620361328125
0 0 0.0295637845993042
0 0 0.0015804767608642578
0 0 0.08426368236541748
0 0 0.0022301673889160156
0 0 0.0396575927734375
0 0 0.16669774055480957
0 0 0.38139867782592773
0 0 0.0026443004608154297
0 0 0.0015878677368164062
0 0 0.33897674083709717
0 0 0.06833648681640625
0 0 0.016819477081298828
0 0 0.1688627004623413
0 0 0.001756429672241211
0 0 0.33368730545043945
0 0 0.0033261775970458984
0 0 0.669570803642273
0 0 0.2246342897415161
0 0 0.1008756160736084
0 0 0.003137350082397461
0 0 2.348452091217041
0 0 0.6228768825531006
0 0 0.03341221809387207
0 0 0.002418994903564453
0 0 0.6033383011817932
0 0 0.18826401233673096
0 0 0.12150537967681885
0 0 1.3897298574447632
0 0 0.06809365749359131
0 0 1.129695177078247
0 0 0.45339953899383545
0 0 0.6708025932312012
0 0 0.49734437465667725
0 0 0.399263143

0 0 0.32270240783691406
0 0 0.0030448436737060547
0 0 0.4376249313354492
0 0 0.4032970666885376
0 0 0.2770146131515503
0 0 0.16938769817352295
0 0 0.31761622428894043
0 0 0.24478363990783691
0 0 0.002081632614135742
0 0 0.0016818046569824219
0 0 0.40713560581207275
0 0 0.10796821117401123
0 0 0.003748655319213867
0 0 0.2145087718963623
0 0 0.586054801940918
0 0 0.0015368461608886719
0 0 0.0019028186798095703
0 0 0.0020656585693359375
0 0 0.7235502004623413
0 0 0.2175748348236084
0 0 0.0033767223358154297
0 0 0.7946690320968628
0 0 0.6291278600692749
0 0 1.8567897081375122
0 0 0.18004679679870605
0 0 0.0031893253326416016
0 0 0.010686874389648438
0 0 0.007958412170410156
0 0 0.056592464447021484
0 0 0.006292104721069336
0 0 0.003860950469970703
0 0 0.19370508193969727
0 0 0.10476434230804443
0 0 0.012885332107543945
0 0 0.005579471588134766
0 0 0.6512798070907593
0 0 0.9217571020126343
0 0 2.1087646484375
0 0 0.012727499008178711
0 0 0.005349159240722656
0 0 0.3735954165458679
0 0 0.160

0 0 0.8697285652160645
0 0 0.2580406665802002
0 0 0.589957594871521
0 0 0.002165555953979492
0 0 0.003406047821044922
0 0 0.002753734588623047
0 0 0.2181774377822876
0 0 0.7271771430969238
0 0 0.39799314737319946
0 0 0.8437880873680115
0 0 0.3063725233078003
0 0 0.0026350021362304688
0 0 0.002439737319946289
0 0 0.7670132517814636
0 0 0.887831449508667
0 0 0.04084420204162598
0 0 0.0014336109161376953
0 0 0.002562999725341797
0 0 0.001863241195678711
0 0 1.4975576400756836
0 0 0.8220387697219849
0 0 0.9172977209091187
0 0 0.31575167179107666
0 0 0.0029664039611816406
0 0 0.0021584033966064453
0 0 0.0017590522766113281
0 0 0.01793694496154785
0 0 0.004716157913208008
0 0 1.389694333076477
0 0 0.1465519666671753
0 0 0.675450325012207
0 0 0.04027295112609863
0 0 0.07558071613311768
0 0 0.09583401679992676
0 0 1.1142964363098145
0 0 0.22111546993255615
0 0 0.0032689571380615234
0 0 0.700685977935791
0 0 0.005190610885620117
0 0 0.006393909454345703
0 0 0.07182061672210693
0 0 0.00616717338

0 0 0.2381356954574585
0 0 0.2104811668395996
0 0 0.0013911724090576172
0 0 0.0022492408752441406
0 0 0.001880645751953125
0 0 0.16871237754821777
0 0 0.18044805526733398
0 0 0.6662781238555908
0 0 0.002007007598876953
0 0 0.0019850730895996094
0 0 0.1818755865097046
0 0 0.11516702175140381
0 0 0.0013899803161621094
0 0 0.0021491050720214844
0 0 0.16221237182617188
0 0 0.1463383436203003
0 0 0.8012810349464417
0 0 0.09962379932403564
0 0 0.09534811973571777
0 0 0.2378239631652832
0 0 0.28389406204223633
0 0 1.6640722751617432
0 0 1.5676352977752686
0 0 0.10360395908355713
0 0 0.28214871883392334
0 0 0.0659637451171875
0 0 0.02221083641052246
0 0 0.21113348007202148
0 0 1.5782525539398193
0 0 0.46209365129470825
0 0 0.2172691822052002
0 0 0.19977331161499023
0 0 0.0016293525695800781
0 0 0.11600875854492188
0 0 0.29134631156921387
0 0 0.4200989007949829
0 0 0.0019063949584960938
0 0 0.0015392303466796875
0 0 0.3249952793121338
0 0 0.4541494846343994
0 0 0.24142682552337646
0 0 1.3536648

KeyboardInterrupt: 


## 評価

In [ ]:
real_data_num, ok =0, 0
net.eval()

with torch.no_grad():
    for i in range(len(x_test)):
        x = torch.LongTensor(x_test[i].unsqueeze(0).to(device))
        ans =  net(x)
        ans1 = torch.argmax(ans,dim=1).item()
        if (ans1 == y_test[i]):
            ok += 1
        real_data_num += 1
print(ok, read_data_num, ok,/real_data_num)
